In [1]:
import random

import cv2
import numpy as np

Download images from directory.

In [2]:
from image import read_image, show_image

In [3]:
images, image_names = read_image("InputImages")

In [4]:
image_num = 1
image = images[image_num]
image_names[image_num]

'1633601797_20-p-foto-doma-snizu-vverkh-foto-33.jpg'

In [5]:
show_image("OutputImage", image)

# Finding Vanishing Points

In [6]:
from line import get_lines, draw_line

In [7]:
lines = get_lines(image)
len(lines)

714

Plot lines.

In [11]:
ouput_image = image.copy()
# Getting the lines form the image
for line in lines:
    r = random.randint(0, 256)
    g = random.randint(0, 256)
    b = random.randint(0, 256)
    draw_line(ouput_image, line, (r, g, b))
show_image("OutputImage", ouput_image)

Cluster lines to find vanishing points.

In [ ]:
from clustering import cluster_lines

In [ ]:
n_clusters = 3

In [ ]:
clusters = cluster_lines(lines, n_clusters=n_clusters)
clusters

Show image with clustered lines and find vanishing points.

In [ ]:
from homography import get_vanishing_point, get_distant_points

In [ ]:
vanishing_points = [get_vanishing_point(clusters[c]) for c in clusters]
vanishing_points

In [ ]:
colors = [(random.randint(0, 256), random.randint(0, 256), random.randint(0, 256)) for _ in range(n_clusters)]

output_image = image.copy()
for c in clusters:
    vp = vanishing_points[c]
    print(vp, colors[c])
    for i, line in enumerate(clusters[c]):
        draw_line(output_image, line, colors[c])
    cv2.circle(output_image, (int(vp[0]), int(vp[1])), 10, colors[c], -1)

show_image("OutputImage", output_image)  

In [ ]:
two_points, _ = get_distant_points(vanishing_points)
two_points

# Estimate intrinsic camera properties

In [ ]:
width, height, _ = image.shape

In [ ]:
diagonal = np.sqrt(width ** 2 + height ** 2)

In [ ]:
f_x = f_y = 0.75 * diagonal

In [ ]:
c_x = width / 2
c_y = width / 2

In [ ]:
K = np.array([
    [f_x, 0, c_x],
    [0, f_y, c_y],
    [0, 0, 1],
])

In [ ]:
# Estimate intrinsic camera properties

width, height, _ = image.shape

diagonal = np.sqrt(width ** 2 + height ** 2)

f_x = f_y = 0.75 * diagonal

c_x = width / 2
c_y = width / 2

K = np.array([
    [f_x, 0, c_x],
    [0, f_y, c_y],
    [0, 0, 1],
])

# Calculate Homography

In [ ]:
V_h = np.linalg.inv(K) @ [*two_points[0], 1]
V_v = np.linalg.inv(K) @ [*two_points[1], 1]
V_h, V_v

In [ ]:
CX = V_h / np.linalg.norm(V_h)
CY = V_v / np.linalg.norm(V_v)
CX, CY

In [ ]:
CZ = np.cross(CX, CY)
CZ

In [ ]:
if CZ[2] < 0:
    CY = -1 * CY
    CZ = -1 * CZ

In [ ]:
if abs(CX.dot(CY)) > 0.1:
    CYY = np.cross(CZ, CX)
else:
    CYY = CY

In [ ]:
R = np.array([
    CX, CYY, CZ,
])
R

In [ ]:
beta = np.arcsin(
    np.linalg.norm(np.cross(CX, CYY)) / (
    np.linalg.norm(CX) * np.linalg.norm(CYY)
    )
)
beta

In [ ]:
A = np.array([
    [1, -np.cos(beta)/np.sin(beta), 0],
    [0, 1/np.sin(beta), 0],
    [0, 0, 1],
])
A

In [ ]:
H = K @ A @ R @ np.linalg.inv(K)
H

In [ ]:
output_image = cv2.warpPerspective(image, H, (image.shape[1], image.shape[0] * 5))

In [ ]:
show_image("OutputImage", output_image)